# Informe Final de la Prueba Técnica 1

**Autor:** Isabel Castrillón Acosta

**Fecha:** 25-06-2025

**Descripción del Proyecto:**


# 1. Explicación del Caso

# 2. Supuestos

# 3. Formas de resolver el caso y la opcion elegida

# 4. Resultados del análisis de los datos y los modelos

## 4.1 Librerias

In [1]:
import pandas as pd
import seaborn as sns
import scikit-learn as skl
import matplotlib.pyplot as plt


SyntaxError: invalid syntax (4037808582.py, line 3)

# 5. Futuras mejoras

# 6. Apreciaciones y comentarios del caso (opcional)